Read in raw data

In [2]:
import pandas as pd

In [27]:
iri_path =  '/Users/sophierand/RichContextMetadata/metadata/20190726_usda_iri/generate_metadata/IRI Sample.xlsx'
iri_linkages = pd.read_excel(iri_path)
iri_linkages = iri_linkages.drop_duplicates()

Import datasets list

In [6]:
import json

In [7]:
datasets_path = '/Users/sophierand/RCDatasets/datasets.json'

with open(datasets_path) as json_file:
    datasets = json.load(json_file)

Dimensions url

In [34]:
import dimensions_search_api_client as dscli

In [35]:
def connect_ds_api(username,password):
    api_client = dscli.DimensionsSearchAPIClient()
    api_client.set_max_in_items( 100 )
    api_client.set_max_return( 1000 )
    api_client.set_max_overall_returns( 50000 )
    api_client.set_username( username )
    api_client.set_password( password )
    return api_client

In [36]:
import configparser
CONFIG = configparser.ConfigParser()
CONFIG.read("dimensions.cfg")

['dimensions.cfg']

In [37]:
api_client = connect_ds_api(username= CONFIG.get('DEFAULT','username'),password = CONFIG.get('DEFAULT','password'))

API credentials have been set


In [38]:
def run_doi_search(doi_id,api_client):
    doi_search_string = 'search publications where doi = "{}" return publications[all] limit 1'.format(doi_id)
    doi_response = api_client.execute_query( query_string_IN=doi_search_string )
    publication_metadata_full = doi_response['publications']
    if publication_metadata_full:
        publication_metadata = publication_metadata_full[0]
        return publication_metadata


In [39]:
doi = '10.1177/0042098017740285'
a = run_doi_search(doi,api_client)

In [41]:
a['linkout']

'https://www.economics.uci.edu/research/wp/1718/17-18-04.pdf'

In [45]:
pub_list = []
for i, r in iri_linkages.iterrows():
    dataset_id = [d['id'] for d in datasets if d['title'].lower() == r['dataset_name'].lower()]
    dimensions_return = run_doi_search(r['doi'],api_client)
    pub_dict = {'title':r['title'],'journal':r['journal'],'related_dataset':dataset_id,'doi':r['doi']}
    try:
        link = dimensions_return['linkout']
        pub_dict.update({'url':link})
    except:
        pass
    pub_list.append(pub_dict)

In [55]:
import os
json_pubs_name = '20190626_iri_usda_publications.json'
pubs_paths =[os.path.join(os.getcwd(), '..','results/',json_pubs_name)
,os.path.join('/Users/sophierand/RichContextMetadata/publications/',json_pubs_name)]


In [56]:
for p in pubs_paths:
    with open(p, 'w') as outfile:
        json.dump(pub_list, outfile,indent=2)
